In [ ]:
print("Testing Notebook") 

Testing Notebook


In [ ]:
from fasthtml.common import *
import asyncio # Ensure asyncio is imported
import re # Ensure re is imported
from urllib.parse import quote_plus # For URL encoding parameters

# This link will be used in the route
wellness_journal_css_link = Link(rel='stylesheet', href='/css/wellness_journal.css')
google_material_icons_link = Link(href="https://fonts.googleapis.com/icon?family=Material+Icons", rel="stylesheet")
wellness_enhancements_js_link = Script(src="/js/wellness_enhancements.js", defer=True)


# Raw SVG string for the filter definition (from fasthtml_raw_scan_line_add_py)
scan_line_svg_defs_raw_string = """
<defs>
  <filter id="scanGlowGreenAnimation">
    <feGaussianBlur stdDeviation="1.2" result="coloredBlur"/>
    <feMerge>
      <feMergeNode in="coloredBlur"/>
      <feMergeNode in="SourceGraphic"/>
    </feMerge>
  </filter>
</defs>
"""

# Raw SVG string for the scan line group itself
scan_line_group_raw_string = """
<g id="scanLineAnimationGroupRaw" style="display: none;">
  <line id="scanLineAnimationElementRaw" x1="10" y1="0" x2="90" y2="0" stroke="#10B981" stroke-width="2" stroke-opacity="0.8" filter="url(#scanGlowGreenAnimation)" stroke-dasharray="5 3">
    <animate attributeName="strokeOpacity" values="0.8;0.3;0.8" dur="1.2s" repeatCount="indefinite" />
  </line>
  <rect id="scanLineHighlightAnimationElementRaw" x="10" y="-2.5" width="80" height="5" fill="#10B981" opacity="0.15" filter="url(#scanGlowGreenAnimation)" />
</g>
"""

# --- Main Body SVG FT Component ---
# This function will create the main annotated SVG body using FastHTML components

def create_main_anatomy_svg():
    # Define default styles for clarity and to ensure they are applied
    default_body_fill = "#97979795"  # Light grey for external parts
    default_body_stroke = "#333333" # Dark grey/black outline
    default_stroke_width = "1px"

    internal_organ_fill = "#e57373" # Reddish for lungs
    internal_organ_stroke = "#c62828" # Darker red stroke for lungs
    internal_organ_fill_opacity = "0.7"

    heart_fill = "#d32f2f" # Specific deep red for heart
    heart_stroke = internal_organ_stroke # Consistent stroke with other internal organs

    return Svg(
        G( # Head
            Ellipse(cx="100", cy="40", rx="35", ry="30", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="head_group", cls="body-part", data_name="Head", data_info="The head contains the brain, eyes, ears, nose, and mouth. It is crucial for sensory input and cognitive functions."
        ),
        G( # Neck
            Rect(x="90", y="70", width="20", height="15", rx="2", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="neck_group", cls="body-part", data_name="Neck", data_info="The neck connects the head to the torso and houses the cervical spine, esophagus, and trachea."
        ),
        G( # Thorax
            Rect(x="60", y="85", width="80", height="75", rx="10", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="thorax_group", cls="body-part", data_name="Thorax (Chest)", data_info="The thorax, or chest, is the part of the torso between the neck and the abdomen. It houses the heart and lungs."
        ),
        G( # Lungs
            Ellipse(cx="85", cy="115", rx="18", ry="25", fill=internal_organ_fill, stroke=internal_organ_stroke, stroke_width=default_stroke_width, fill_opacity=internal_organ_fill_opacity),
            Ellipse(cx="115", cy="115", rx="18", ry="25", fill=internal_organ_fill, stroke=internal_organ_stroke, stroke_width=default_stroke_width, fill_opacity=internal_organ_fill_opacity),
            id="lungs_group", cls="body-part internal-organ", data_name="Lungs", data_info="The lungs are the primary organs of the respiratory system, responsible for oxygenating blood."
        ),
        G( # Heart
            Ellipse(cx="100", cy="120", rx="15", ry="15", style=f"fill: {heart_fill};", stroke=heart_stroke, stroke_width=default_stroke_width, fill_opacity=internal_organ_fill_opacity), # Retain specific heart fill, add stroke
            id="heart_group", cls="body-part internal-organ", data_name="Heart", data_info="The heart is a muscular organ that pumps blood throughout the body via the circulatory system."
        ),
        G( # Abdominal and Pelvic Region
            Rect(x="60", y="160", width="80", height="60", rx="10", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="abdominal_pelvic_region_group", cls="body-part", data_name="Abdominal and Pelvic Region", data_info="This region includes the abdomen and pelvis. It houses major organs of the digestive, urinary, and reproductive systems (e.g., stomach, intestines, liver, kidneys, bladder, reproductive organs)."
        ),
        G( # Left Shoulder
            Circle(cx="60", cy="95", r="12", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="left_shoulder_group", cls="body-part", data_name="Left Shoulder", data_info="The left shoulder joint connects the left arm to the torso."
        ),
        G( # Right Shoulder
            Circle(cx="140", cy="95", r="12", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="right_shoulder_group", cls="body-part", data_name="Right Shoulder", data_info="The right shoulder joint connects the right arm to the torso."
        ),
        G( # Left Arm
            Rect(x="25", y="95", width="30", height="70", rx="5", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="left_arm_group", cls="body-part", data_name="Left Arm", data_info="The left arm is used for reaching, grasping, and interacting with the environment."
        ),
        G( # Right Arm
            Rect(x="145", y="95", width="30", height="70", rx="5", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="right_arm_group", cls="body-part", data_name="Right Arm", data_info="The right arm, similar to the left, facilitates interaction and manipulation of objects."
        ),
        G( # Left Hand
            Ellipse(cx="40", cy="170", rx="12", ry="8", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="left_hand_group", cls="body-part", data_name="Left Hand", data_info="The left hand is at the end of the left arm, used for manipulation."
        ),
        G( # Right Hand
            Ellipse(cx="160", cy="170", rx="12", ry="8", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="right_hand_group", cls="body-part", data_name="Right Hand", data_info="The right hand is at the end of the right arm, used for manipulation."
        ),
        G( # Left Leg
            Rect(x="70", y="220", width="25", height="70", rx="5", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="left_leg_group", cls="body-part", data_name="Left Leg", data_info="The left leg supports the body and enables locomotion."
        ),
        G( # Right Leg
            Rect(x="105", y="220", width="25", height="70", rx="5", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="right_leg_group", cls="body-part", data_name="Right Leg", data_info="The right leg provides support and mobility."
        ),
        G( # Left Foot
            Ellipse(cx="82.5", cy="295", rx="15", ry="8", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="left_foot_group", cls="body-part", data_name="Left Foot", data_info="The left foot is at the end of the left leg, crucial for standing and walking."
        ),
        G( # Right Foot
            Ellipse(cx="117.5", cy="295", rx="15", ry="8", fill=default_body_fill, stroke=default_body_stroke, stroke_width=default_stroke_width),
            id="right_foot_group", cls="body-part", data_name="Right Foot", data_info="The right foot is at the end of the right leg, crucial for standing and walking."
        ),
        # Main SVG attributes
        id="mainAnatomySvgFT",
        viewBox="0 -70 200 450", # viewBox adjusted for centering
        xmlns="http://www.w3.org/2000/svg",
        cls="object-contain w-full h-full p-2",
        style="position: relative; z-index: 2; background-color: #f8fafc; border: 1px solid #e2e8f0;"
    )
main_anatomy_svg_ft = create_main_anatomy_svg()


pre_configured_conversation_history = [
    'Human Message: "I’ve had a sore throat for 3 days."',
    'AI Message: "Do you also have a fever or cough?"',
    'Human Message: "Yes, I have a mild fever but no cough."',
    'AI Message: "Are you experiencing any difficulty swallowing or swollen glands?"',
    'Human Message: "Swallowing is painful, and my neck feels tender."',
    'AI Message: "It may be a throat infection; I recommend seeing a doctor for a throat swab."'
]

# Body scanner action commands list
body_scanner_action_commands_list = [
    "START_SCAN", "STOP_SCAN", "Head", "Neck", "Thorax", "Lungs", "Heart",
    "Abdominal and Pelvic Region", "Left Shoulder", "Right Shoulder",
    "Left Arm", "Right Arm", "Left Hand", "Right Hand", "Left Leg",
    "Right Leg", "Left Foot", "Right Foot", "FULL_BODY_GLOW"
]


# MODIFICATION START: Dummy app and rt for context, replace with your actual app initialization
# from fasthtml.core import FastHTML
# app, rt = FastHTML(), FastHTML().route # Replace this with your actual app = FastHTML(...) and rt = app.route
# This is just to make the snippet runnable in isolation for testing.
# In your actual app, you'll have:
# app = FastHTML(...)
# rt = app.route
# For example:
# app = FastHTML(hdrs=(google_material_icons_link, wellness_journal_css_link, wellness_enhancements_js_link))
# rt = app.route
# MODIFICATION END

async def llm_agent_1(user_message: str, conversation_history: list = None): # Added conversation_history
    """
    Simulates an LLM agent processing the user message.
    Returns a structured response after a delay.
    """
    print(f"llm_agent_1 received: {user_message}")
    print(f"llm_agent_1 history: {conversation_history}") # Log history
    await asyncio.sleep(2) # Simulate 2-second delay

    # Determine body_scanner_animation_action_comand
    found_command = "idle" # Default command
    for command in body_scanner_action_commands_list:
        # Simple case-insensitive keyword matching
        if re.search(r'\b' + re.escape(command) + r'\b', user_message, re.IGNORECASE):
            found_command = command
            break

    # For START_SCAN and STOP_SCAN, ensure they are specific enough if they are keywords in body_scanner_action_commands_list
    if "start scan" in user_message.lower():
        found_command = "START_SCAN"
    elif "stop scan" in user_message.lower():
        found_command = "STOP_SCAN"
    elif "full body glow" in user_message.lower() or "glow" in user_message.lower(): # Added "glow" for flexibility
        found_command = "FULL_BODY_GLOW"

    # Simulate a response based on the user message
    ai_text_response = f"AI processed: '{user_message}'. My predefined answer is Hello World."
    if "sore throat" in user_message.lower():
        ai_text_response = "Regarding your sore throat, do you also have a fever or cough?"
    elif "headache" in user_message.lower():
        ai_text_response = "For headaches, it's helpful to know the location and type of pain. Can you describe it further?"


    response_data = {
        "ai_response": ai_text_response, # Dynamic AI response
        "body_scanner_animation_action_comand": found_command,
        "trigger_wellness_journal_data_listener": "START", # This can be made dynamic too
        "output_conversation_history": conversation_history + [f"Human Message: \"{user_message}\"", f"AI Message: \"{ai_text_response}\""] if conversation_history else [f"Human Message: \"{user_message}\"", f"AI Message: \"{ai_text_response}\""]
    }
    print(f"llm_agent_1 output: {response_data}")
    return response_data

def unified_ui_controller_for_chat_window_and_body_scanner(llm_data: dict): # Removed user_message_text as it's in llm_data or history
    """
    Processes LLM data to generate UI updates (primarily AI chat bubble).
    """
    ai_response_text = llm_data.get("ai_response", "Sorry, I encountered an issue.")

    # Generate AI chat bubble FT component
    ai_chat_bubble = Div(
        Div(
            Div( # Avatar
                Div(
                    Span("smart_toy", cls="material-icons emoji-icon"),
                    cls="bg-transparent text-neutral-content rounded-full w-8 h-8 text-sm flex items-center justify-center"
                ),
                cls="avatar placeholder p-0 w-8 h-8 rounded-full mr-2 bg-base-300"
            ),
            Div( # Bubble content
                P(ai_response_text, cls="text-sm leading-relaxed chat-message-text"),
                cls="chat-bubble chat-bubble-neutral bg-base-300 text-base-content rounded-bl-none shadow-md"
            ),
            cls="flex items-end max-w-xs sm:max-w-md md:max-w-lg"
        ),
        cls="flex justify-start chat-message-container animate-slideUp",
    )

    # Print other data (for future implementation)
    print(f"Unified UI Controller - Body Scanner Command: {llm_data.get('body_scanner_animation_action_comand')}")
    print(f"Unified UI Controller - Wellness Journal Trigger: {llm_data.get('trigger_wellness_journal_data_listener')}")
    # print(f"Unified UI Controller - Conversation History: {llm_data.get('output_conversation_history')}") # History is now part of llm_data

    return ai_chat_bubble

# --- Modified route ---
@rt("/send_chat_message")
async def handle_send_chat_message_htmx(req): # Renamed for clarity, though path remains the same
    form_data = await req.form()
    user_message_text = form_data.get("chatInput", "").strip()
    
    # MODIFICATION: Collect conversation history from hidden inputs if they exist
    # This part depends on how you decide to send history.
    # For now, we'll assume it might come as multiple 'conversation_history[]' fields
    # or similar. If not sent, llm_agent_1 will use its default/pre_configured.
    # A robust way would be to have JavaScript collect this and put it into a single field,
    # but per "no JS" rule, we rely on multiple form fields or server-side session.
    # For this example, let's assume `llm_agent_1` handles history internally or via session
    # if not directly passed. Or, we pass it to the second call.

    if not user_message_text:
        return Textarea(id="chatInput", name="chatInput", placeholder="Describe your symptoms here...", cls="textarea textarea-bordered flex-grow resize-none scrollbar-thin", rows="1", style="min-height: 44px; max-height: 120px;", hx_swap_oob="true")

    # 1. User message chat bubble
    user_chat_bubble = Div(
        Div(
            Div( # User Avatar and Bubble
                Div( # Bubble content
                    P(user_message_text, cls="text-sm leading-relaxed chat-message-text"),
                    cls="chat-bubble chat-bubble-primary user-message-gradient rounded-br-none shadow-md"
                ),
                Div( # Avatar
                    Div(
                        Span("person", cls="material-icons emoji-icon"),
                        cls="bg-transparent text-neutral-content rounded-full w-8 h-8 text-sm flex items-center justify-center"
                    ),
                    cls="avatar placeholder p-0 w-8 h-8 rounded-full ml-2 user-message-gradient order-last" # order-last to put avatar after bubble
                ),
                cls="flex items-end max-w-xs sm:max-w-md md:max-w-lg flex-row-reverse" # flex-row-reverse for user
            ),
            cls="flex justify-end chat-message-container animate-slideUp" # justify-end for user
        )
    )

    # 2. Placeholder for AI response that triggers a second HTMX request
    # We need to pass the user_message_text to the AI agent.
    # For simplicity, we'll URL encode it.
    # If conversation history needs to be passed, it would also need to be encoded.
    # For now, `llm_agent_1` uses `pre_configured_conversation_history` or its own logic.
    user_message_encoded = quote_plus(user_message_text)
    
    # Placeholder with a simple loading text. CSS can style this better.
    ai_response_placeholder = Div(
        P("Akasi.ai is thinking...", cls="text-sm italic text-base-content/70 p-2 text-center"), # Simple loading text
        id=f"ai-placeholder-{asyncio.locks. μεταξύ(0, 1000000)}", # Unique ID for the placeholder
        hx_get=f"/get_ai_response_htmx?user_message={user_message_encoded}", # Pass user message
        hx_trigger="load", # Trigger request immediately when this div is loaded into the DOM
        hx_target="this",
        hx_swap="outerHTML" # The AI response will replace this entire placeholder div
    )

    # 3. OOB swap for clearing the chat input
    cleared_chat_input = Textarea(id="chatInput", name="chatInput", placeholder="Describe your symptoms here...", cls="textarea textarea-bordered flex-grow resize-none scrollbar-thin", rows="1", style="min-height: 44px; max-height: 120px;", hx_swap_oob="true")
    
    # Return user bubble, AI placeholder (which triggers AI fetch), and cleared input
    return user_chat_bubble, ai_response_placeholder, cleared_chat_input

# --- NEW HTMX route for getting AI response ---
@rt("/get_ai_response_htmx")
async def get_ai_response_htmx_route(req, user_message: str): # user_message from query param
    # In a more complex scenario, you might also pass/retrieve conversation history here
    # For now, llm_agent_1 uses its internal/pre_configured history logic
    # If you were to pass history, ensure it's handled securely and robustly (e.g., via session or encrypted client-side storage if JS were allowed)
    
    # For this example, we'll pass the current pre_configured_conversation_history
    # A real app would build history dynamically.
    current_history = pre_configured_conversation_history.copy() # Use a copy

    llm_output = await llm_agent_1(user_message, conversation_history=current_history) # Pass history
    ai_chat_bubble = unified_ui_controller_for_chat_window_and_body_scanner(llm_output)
    
    # This response will replace the placeholder.
    # We can also add HX-Reswap here if needed, or HX-Scroll-Bottom
    # For auto-scroll, the #messagesArea should have `hx-swap="beforeend show:bottom"`
    # or this response can add `HX-Scroll-Bottom: #messagesArea` if the placeholder
    # itself is not inside a scroll-managed container in a way that `show:bottom` works.
    # Since the placeholder is swapped with outerHTML, the scroll should ideally be
    # handled by the container of these messages.
    return ai_chat_bubble


@rt('/onboarding/wellness-journal')
def get(auth=None): # Made auth optional for now if not strictly used
    # if auth is None: # Uncomment if auth is required
    #     return RedirectResponse('/login', status_code=303)

    scan_line_svg_overlay_raw = Svg(
        NotStr(scan_line_svg_defs_raw_string),
        NotStr(scan_line_group_raw_string),
        id="scanLineSvgOverlayRaw",
        viewBox="0 0 100 200",
        width="100%",
        height="100%",
        style="position: absolute; top: 0; left: 0; z-index: 3; pointer-events: none;"
    )

    chatbox_header = Header(
        H1("Akasi.ai Chat", cls="text-lg sm:text-xl font-semibold text-center"),
        Div(
            Button(
                Span("refresh", cls="material-icons emoji-icon mr-1"), " Clear Chat",
                id="clearChatButton",
                cls="btn btn-xs bg-white/20 hover:bg-white/30 border-none flex items-center gap-1",
                title="Clear chat history"
            ),
            # Removed Add AI/User Bubble buttons for brevity, can be added back if needed
            cls="flex space-x-2 mt-1.5"
        ),
        id="chatboxHeader",
        cls="p-3.5 sm:p-4 shadow-md primary-green-gradient flex flex-col items-center"
    )

    # MODIFICATION: Added hx-ext="ws" if you plan to use websockets for real-time later.
    # For scrolling, if the placeholder method works, the scroll is handled by the swap of the placeholder.
    # Alternatively, the main #messagesArea can have an hx-on::oob-after-swap="this.scrollTop = this.scrollHeight"
    # but that's JS. Let's rely on HTMX's `show:bottom` or server-sent scroll commands.
    messages_area = Div(
        # Initial messages can go here if any
        Div( # Example initial AI message
            Div(
                Div( # Avatar
                    Div(
                        Span("smart_toy", cls="material-icons emoji-icon"),
                        cls="bg-transparent text-neutral-content rounded-full w-8 h-8 text-sm flex items-center justify-center"
                    ),
                    cls="avatar placeholder p-0 w-8 h-8 rounded-full mr-2 bg-base-300"
                ),
                Div( # Bubble content
                    P("Hello! How can I help you with your wellness today?", cls="text-sm leading-relaxed chat-message-text"),
                    cls="chat-bubble chat-bubble-neutral bg-base-300 text-base-content rounded-bl-none shadow-md"
                ),
                cls="flex items-end max-w-xs sm:max-w-md md:max-w-lg"
            ),
            cls="flex justify-start chat-message-container animate-slideUp",
        ),
        id="messagesArea",
        cls="flex-grow p-3 sm:p-4 space-y-3 overflow-y-auto bg-base-200 scrollbar-thin"
        # We want new content to scroll this div.
        # The form's hx-target="#messagesArea" and hx-swap="beforeend show:bottom" should handle this.
    )

    chat_input_area = Div(
        Div(id="stagedAttachmentsContainer", cls="mb-2 p-2 border border-base-300 rounded-lg bg-base-200 max-h-32 overflow-y-auto scrollbar-thin space-y-2 hidden"),
        Div(
            Div( # Attachment dropdown
                Button(
                    Span("add", cls="material-icons emoji-icon text-2xl"),
                    id="attachmentButton", tabindex="0", role="button", title="Attach files",
                    cls="btn btn-ghost btn-circle text-primary"
                ),
                Div(
                    Button(Span("image", cls="material-icons emoji-icon"), " Attach Image(s)", id="attachImageButton", cls="btn btn-sm btn-ghost justify-start gap-2"),
                    Button(Span("article", cls="material-icons emoji-icon"), " Attach Document(s)", id="attachDocumentButton", cls="btn btn-sm btn-ghost justify-start gap-2"),
                    id="attachmentOptionsContent", tabindex="0",
                    cls="dropdown-content z-[20] menu p-2 shadow bg-base-100 rounded-box w-56 mb-2"
                ),
                id="attachmentOptionsDropdownContainer", cls="dropdown dropdown-top"
            ),
            Form(
                Input(type="file", multiple=True, id="fileInput", name="files", cls="hidden"),
                Textarea(id="chatInput", name="chatInput",
                         placeholder="Describe your symptoms here...",
                         cls="textarea textarea-bordered flex-grow resize-none scrollbar-thin",
                         rows="1", style="min-height: 44px; max-height: 120px;",
                         # MODIFICATION: Trigger on Enter key for textarea
                         hx_trigger="keyup[key=='Enter' && !shiftKey && !metaKey && !ctrlKey && !altKey] from:this", # Trigger on Enter
                         hx_post="/send_chat_message", # This will now also be triggered by Enter
                         hx_target="#messagesArea",
                         # MODIFICATION: Added show:bottom to scroll on new message append
                         hx_swap="beforeend show:bottom",
                         hx_encoding="multipart/form-data" # Keep if file uploads are intended
                        ),
                Button( # Send Button
                    Span("send", cls="material-icons emoji-icon text-xl", id="sendButtonIcon"), # Added ID for icon
                    Span("Loading...", cls="loading-spinner-material hidden", id="sendButtonSpinner"), # Hidden by default
                    id="sendButton",
                    type="submit", # Submit the form
                    cls="btn btn-primary btn-circle"
                    # HTMX attributes are now on the Form element for Enter key,
                    # but button click will also submit the form.
                ),
                # HTMX attributes moved to the Textarea for Enter key submission,
                # and also kept on the form for general submission logic.
                # The form itself can also have these hx attributes.
                # Let's ensure the form itself handles the HTMX POST for clarity and button clicks.
                # The textarea's hx-trigger will make the form submit.
                id="chatForm", # Added ID to the form
                hx_post="/send_chat_message",
                hx_target="#messagesArea",
                hx_swap="beforeend show:bottom", # Ensures scroll after new elements are appended
                hx_encoding="multipart/form-data",
                cls="flex items-end space-x-2 sm:space-x-3 flex-grow"
            ),
            cls="flex items-end space-x-2 sm:space-x-3"
        ),
        cls="bg-base-100 p-3 sm:p-4 shadow-inner border-t border-base-300"
    )

    left_panel_chatbox = Div(
        Div(
            chatbox_header,
            messages_area,
            Div(id="messagesEndRef", cls="h-0"), # Helper for potential JS scroll, but we aim for HTMX
            chat_input_area,
            id="chatboxRoot",
            cls="flex flex-col h-full bg-base-100 text-base-content rounded-lg shadow-xl overflow-hidden border border-base-300"
        ),
        cls="w-full md:w-2/5 lg:w-1/3 h-1/2 md:h-full flex flex-col"
    )

    scanner_visual_container = Div(
        main_anatomy_svg_ft,
        scan_line_svg_overlay_raw,
        cls="border-2 border-emerald-500 rounded-lg bg-white/10 flex items-center justify-center relative overflow-hidden",
        style=f"width: 350px; height: 500px;"
    )

    scanner_main_area_content = Div(
        Div(
            Button("Start Scan", id="debugStartScanButton", cls="btn primary-green-gradient",
                   hx_get="/trigger_body_scan_animation_script", hx_target="#script_runner_area", hx_swap="innerHTML"),
            Button("Stop Scan", id="debugStopScanButton", cls="btn primary-green-gradient",
                   hx_get="/trigger_stop_body_scan_script", hx_target="#script_runner_area", hx_swap="innerHTML"),
            Button("Narrow Scan", id="debugNarrowScanButton", cls="btn primary-green-gradient",
                   hx_get="/js_show_narrow_scan_modal", hx_target="#script_runner_area", hx_swap="innerHTML"),
            Button("Full Body Glow", id="debugFullBodyGlowButton", cls="btn primary-green-gradient",
                   hx_get="/trigger_body_glow_script", hx_target="#script_runner_area", hx_swap="innerHTML"),
            cls="grid grid-cols-2 sm:grid-cols-4 gap-2 w-full max-w-md px-2"
        ),
        Div(
            scanner_visual_container,
            Div(
                Div(
                    P("Scanner idle. Describe symptoms or restart. ", id="scannerStatusText", cls="text-base-content text-sm"),
                    Span("💤", id="scannerStatusIconContainer", cls="mt-1 text-xl text-primary h-6 w-6 flex items-center justify-center"),
                    id="scannerStatusContainer",
                    cls="flex flex-col items-center justify-center h-full"
                ),
                cls="text-center h-10 w-full max-w-xs mb-3 mt-6"
            ),
            cls="flex flex-col items-center justify-center flex-grow w-full "
        ),
        Div(
            Button("Restart Scan", id="restartScanButton", cls="btn btn-outline btn-sm w-full mb-2"),
            Button(
                Span("menu_book", cls="material-icons emoji-icon"), " Finish Wellness Journal",
                id="finishJournalButton",
                cls="btn btn-sm w-full flex items-center justify-center gap-2 primary-green-gradient"
            ),
            cls="w-full max-w-xs flex flex-col items-center"
        ),
        cls="flex flex-col flex-grow h-full p-4 md:p-6 items-center justify-between relative"
    )

    journal_entries_list_items = [
        Div(
            Div(
                H3("Headache", cls="font-medium text-base-content/90 text-sm"),
                Span("Medium", cls="text-xs font-semibold px-2 py-0.5 rounded-full border text-yellow-700 border-yellow-400 bg-yellow-100"),
                cls="flex justify-between items-start mb-1"
            ),
            P("Dull ache reported across the forehead area.", cls="text-base-content/80 text-xs mb-1.5 leading-relaxed"),
            P("5/17/2025", cls="text-base-content/70 text-xs text-right"),
            cls="bg-base-100/80 backdrop-blur-sm rounded-lg p-3 shadow-md hover:shadow-lg transition-shadow border border-base-300/80"
        ),
        # ... other journal entries ...
        Div(
            Span("warning", id="noJournalIconContainer", cls="material-icons mb-2 text-4xl"),
            P("No entries yet.", cls="text-sm"),
            P("Symptoms you describe will appear here.", cls="text-xs mt-1"),
            id="noJournalEntries",
            cls="flex flex-col items-center justify-center h-full text-base-content/70 text-center py-6",
            style="display: none;" # Initially hidden
        )
    ]

    journal_entries_panel = Div(
        Div(
            H2("Wellness Journal", cls="text-md font-semibold text-center flex-grow"),
            Button(
                Span("add", cls="material-icons emoji-icon text-lg"),
                id="addManualEntryButton",
                cls="btn btn-xs btn-circle btn-ghost",
                title="Add Manual Entry"
            ),
            cls="p-3.5 border-b border-primary/60 primary-green-gradient flex justify-between items-center"
        ),
        Div(
            *journal_entries_list_items,
            id="journalEntriesList",
            cls="flex-grow overflow-y-auto p-3 space-y-2.5 scrollbar-thin"
        ),
        Div(
            Button(
                Span("delete_sweep", cls="material-icons emoji-icon mr-1"), " Clear All",
                id="clearAllJournalButton",
                cls="btn btn-xs btn-outline btn-error flex items-center gap-1.5",
                title="Clear all journal entries"
            ),
            id="clearJournalContainer",
            cls="p-2.5 border-t border-primary/30 flex justify-end",
            style="display: flex;"
        ),
        cls="w-full md:w-2/5 lg:w-1/3 h-full border-l border-primary/30 flex flex-col bg-base-100/50"
    )

    right_panel_scanner_journal_root = Div(
        scanner_main_area_content,
        journal_entries_panel,
        id="scannerJournalRoot",
        cls="flex flex-col md:flex-row w-full h-full text-base-content rounded-lg shadow-xl overflow-hidden subtle-green-gradient-bg border border-base-300"
    )

    right_panel_wrapper = Div(
        right_panel_scanner_journal_root,
        cls="w-full md:w-3/5 lg:w-2/3 h-1/2 md:h-full flex"
    )

    page_content = Div(
        left_panel_chatbox,
        right_panel_wrapper,
        cls="flex flex-col md:flex-row h-full text-base-content p-2 sm:p-4 gap-2 sm:gap-4 font-sans"
    )

    narrow_scan_modal = Dialog(
        Div(
            Form( # Close button form
                Button(
                    Span("close", cls="material-icons emoji-icon"),
                    id="closeNarrowScanModalButton", # ID for JS if needed, but form method works
                    cls="btn btn-sm btn-circle btn-ghost absolute right-2 top-2"
                ),
                method="dialog" # Closes the dialog
            ),
            H3("Specify Narrow Scan Target", cls="font-bold text-lg mb-3"),
            Input(type="text", id="narrowScanInput", placeholder="E.g., Head, Lungs, Arm", cls="input input-bordered w-full mb-3"),
            Button("Confirm & Scan", id="confirmNarrowScanButton", cls="btn btn-primary w-full"), # JS will handle this
            cls="modal-box"
        ),
        Form(Button("close"), method="dialog", cls="modal-backdrop"), # Click outside to close
        id="narrowScanModal", cls="modal"
    )

    manual_entry_modal = Dialog(
        Div(
            Form( # Close button form
                Button(
                    Span("close", cls="material-icons emoji-icon"),
                    id="closeManualEntryModalButton",
                    cls="btn btn-sm btn-circle btn-ghost absolute right-2 top-2"
                ),
                method="dialog"
            ),
            H3("Add Manual Journal Entry", cls="font-bold text-lg mb-4"),
            Form( # Actual entry form
                Div(
                    Label("Title / Body Part", For="manualTitle", cls="block text-xs font-medium text-base-content/80 mb-1"),
                    Input(type="text", id="manualTitle", name="title", placeholder="E.g., Headache, Stomach Ache", cls="input input-bordered input-sm w-full", required=True),
                    cls="mb-3"
                ),
                Div(
                    Label("Status / Severity", For="manualStatus", cls="block text-xs font-medium text-base-content/80 mb-1"),
                    Select(
                        Option("Low", value="1"),
                        Option("Medium", value="2"),
                        Option("High", value="3"),
                        id="manualStatus", name="status", cls="select select-bordered select-sm w-full"
                    ),
                    cls="mb-3"
                ),
                Div(
                    Label("Summary / Description", For="manualSummary", cls="block text-xs font-medium text-base-content/80 mb-1"),
                    Textarea(id="manualSummary", name="summary", placeholder="Describe the issue or feeling...", rows="3", cls="textarea textarea-bordered textarea-sm w-full", required=True),
                    cls="mb-3"
                ),
                Div(
                    Label("Date", For="manualDate", cls="block text-xs font-medium text-base-content/80 mb-1"),
                    Input(type="date", id="manualDate", name="date", cls="input input-bordered input-sm w-full", required=True), # JS will set current date
                    cls="mb-4"
                ),
                Div(
                    Button("Cancel", type="button", id="cancelManualEntryButton", cls="btn btn-sm btn-ghost"), # type="button" to prevent form submission
                    Button("Save Entry", type="submit", id="saveManualEntryButton", cls="btn btn-sm btn-primary"),
                    cls="flex justify-end gap-2 modal-action mt-0" # modal-action for DaisyUI styling
                ),
                id="manualEntryForm" # For JS or HTMX targeting if needed later
                # Add hx-post, hx-target etc. here if submitting via HTMX
            ),
            cls="modal-box"
        ),
        Form(Button("close"), method="dialog", cls="modal-backdrop"),
        id="manualEntryModal", cls="modal"
    )

    diary_loading_overlay = Div(
        Span("refresh", id="diaryLoadingIconContainer", cls="material-icons text-emerald-400 mb-6 text-5xl animate-subtle-spin"),
        H2("We are building your health diary...", cls="text-white text-2xl font-semibold mb-3"),
        P(
            "Please wait a moment while Akasi.ai compiles your personalized wellness summary.",
            cls="text-center text-gray-300 text-lg max-w-md"
        ),
        id="diaryLoadingOverlay",
        cls="fixed inset-0 bg-black/70 backdrop-blur-sm z-[100] flex flex-col justify-center items-center p-4 animate-fadeIn",
        style="display: none;"
    )

    toast_container = Div(id="toastContainer", cls="toast toast-top toast-center z-[200]")
    script_runner_area = Div(id="script_runner_area") # For HTMX to inject scripts

    full_page_wrapper = Div(
        page_content,
        narrow_scan_modal,
        manual_entry_modal,
        diary_loading_overlay,
        toast_container,
        script_runner_area,
        cls="h-screen"
    )

    return (
        Title("Wellness Journal - Akasi.ai"),
        google_material_icons_link,
        wellness_journal_css_link,
        wellness_enhancements_js_link,
        full_page_wrapper
    )


# --- HTMX UI ROUTES / FUNCTIONS  ---

@rt("/trigger_body_scan_animation_script")
def get_trigger_body_scan_animation_script():
    js_code = """
    (() => {
        if (typeof startBodyScanAnimation === 'function') {
            startBodyScanAnimation();
        } else {
            console.error('startBodyScanAnimation function not defined in wellness_enhancements.js');
        }
    })();
    """
    return Script(js_code)

@rt("/trigger_stop_body_scan_script")
def get_trigger_stop_body_scan_script():
    js_code = """
    (() => {
        if (typeof stopBodyScanAnimation === 'function') {
            stopBodyScanAnimation();
        } else {
            console.error('stopBodyScanAnimation function not defined in wellness_enhancements.js');
        }
    })();
    """
    return Script(js_code)


@rt("/trigger_body_glow_script")
def get_trigger_body_glow_script():
    js_code = """
    (() => {
        if (typeof toggleBodyGlowEffect === 'function') {
            toggleBodyGlowEffect();
        } else {
            console.error('toggleBodyGlowEffect function not defined in wellness_enhancements.js');
        }
    })();
    """
    return Script(js_code)

@rt("/js_show_narrow_scan_modal")
def js_show_narrow_scan_modal_script():
    js_code = """
    (() => {
        const modal = document.getElementById('narrowScanModal');
        if (modal && typeof modal.showModal === 'function') {
            const narrowInput = document.getElementById('narrowScanInput');
            if (narrowInput) {
                narrowInput.value = '';
            }
            modal.showModal();
        } else {
            console.error('Narrow scan modal or its showModal function not found.');
        }
    })();
    """
    return Script(js_code)

# Make sure to have your app and rt defined, e.g.:
# from fasthtml.core import FastHTML
# app = FastHTML(hdrs=(google_material_icons_link, wellness_journal_css_link, wellness_enhancements_js_link))
# rt = app.route
# serve() # if running this file directly